In [ ]:
# Imports
try:
    from PIL import Image
    from PIL.ExifTags import TAGS, GPSTAGS
except ImportError:
    !pip install pillow defusedxml xmptools
    from PIL import Image
    from PIL.ExifTags import TAGS, GPSTAGS

try:
    import numpy as np
except ImportError:
    !pip install numpy
    import numpy as np

try:
    from pyproj import Transformer
except ImportError:
    !pip install pyproj
    from pyproj import Transformer
    
try:
    import geopandas as gpd
except ImportError:
    !pip install geopandas
    import geopandas as gpd


# Aux Functions e data
def Ry(pitch):
  return np.array([[np.cos(pitch), 0, np.sin(pitch)],
                   [0            , 1,      0        ],
                   [-np.sin(pitch), 0, np.cos(pitch) ]])
def Rx(roll):
  return np.array([[1,        0      ,      0     ],
                   [0, np.cos(roll) , -np.sin(roll)],
                   [0, np.sin(roll), np.cos(roll)]])
def Rz(yaw):
  return np.array([[np.cos(yaw) ,-np.sin(yaw), 0],
                   [np.sin(yaw),np.cos(yaw), 0],
                   [0             ,0        , 1]])
def Rypr( yaw, pitch, roll ):
  # (pitch, roll, yaw)=(phi, omega, kappa) --> sim, talvez, não
  return Rz( np.deg2rad(yaw)) @ Ry(np.deg2rad(pitch)) @ Rx(np.deg2rad(roll))

def get_params(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    exif_dict = {TAGS.get(tag): value for tag, value in exif_data.items()}
    exif_dict.update({GPSTAGS.get(tag): value for tag, value in exif_dict['GPSInfo'].items() if tag in GPSTAGS})

    # Exibir informações EXIF
    XResolution = float(exif_dict.get('XResolution'))
    YResolution = float(exif_dict.get('YResolution'))
    FocalLength = float(exif_dict.get('FocalLength'))
    ExifImageWidth = float(exif_dict.get('ExifImageWidth'))
    ExifImageHeight = float(exif_dict.get('ExifImageHeight'))
    # Exibir informações GPS
    GPSLatitude = exif_dict.get('GPSLatitude')[0] + exif_dict.get('GPSLatitude')[1]/60.0 + exif_dict.get('GPSLatitude')[2]/3600.0
    GPSLatitude = (-1) * GPSLatitude if exif_dict.get('GPSLatitudeRef') == 'S' else GPSLatitude
    GPSLongitude = exif_dict.get('GPSLongitude')[0] + exif_dict.get('GPSLongitude')[1]/60.0 + exif_dict.get('GPSLongitude')[2]/3600.0
    GPSLongitude = (-1) * GPSLongitude if exif_dict.get('GPSLongitudeRef') == 'W' else GPSLongitude
    GPSAltitude = exif_dict.get('GPSAltitude')

    # Exibir informações XMP
    xmp_data = image.getxmp()['xmpmeta']['RDF']['Description']
    GimbalRollDegree = float(xmp_data.get('GimbalRollDegree'))
    GimbalYawDegree = float(xmp_data.get('GimbalYawDegree'))
    GimbalPitchDegree = float(xmp_data.get('GimbalPitchDegree'))
    FlightRollDegree = float(xmp_data.get('FlightRollDegree'))
    FlightYawDegree = float(xmp_data.get('FlightYawDegree'))
    FlightPitchDegree = float(xmp_data.get('FlightPitchDegree'))
    CalibratedFocalLength = float(xmp_data.get('CalibratedFocalLength'))
    CalibratedOpticalCenterX = float(xmp_data.get('CalibratedOpticalCenterX'))
    CalibratedOpticalCenterY = float(xmp_data.get('CalibratedOpticalCenterY'))
    FieldOfView = float(73.7) # from exiftools
    # FieldOfView = float(84) # from dji site
    return (GimbalYawDegree, GimbalPitchDegree, GimbalRollDegree, 
            GPSLongitude, GPSLatitude, GPSAltitude,
            CalibratedFocalLength, CalibratedOpticalCenterX, CalibratedOpticalCenterY, 
            FocalLength,
            ExifImageWidth, ExifImageHeight,
            FieldOfView)

gcp = gpd.read_file("aux_data/gcp.shp")

# (i) Extração das coordenadas do pixels nas duas imagens ($p_1$ e $p_2$ ) correspondete ao GCP conhecido:
p1 = np.array([2456,1052])
p2 = np.array([2498,1938])

# (ii) Extração da matrizes Extrínsicas ($T_1$ e $T_2$) e Intrísica ($K_1$ e $K_2$) dos metadados das imagens:
file1 = 'data/DJI_0019.JPG'
GimbalYawDegree, GimbalPitchDegree, GimbalRollDegree, GPSLongitude, GPSLatitude, GPSAltitude, CalibratedFocalLength, CalibratedOpticalCenterX, CalibratedOpticalCenterY, FocalLength, ExifImageWidth, ExifImageHeight, FieldOfView = get_params(file1)

## Matriz extrínsica T1
R1 = Rypr(GimbalYawDegree, GimbalPitchDegree, GimbalRollDegree)
transformer = Transformer.from_crs("epsg:4326", "epsg:31980")  # WGS84 → ECEF
x, y, z = transformer.transform(GPSLatitude, GPSLongitude, GPSAltitude)  # C in world coordinates
C = np.array([[x],[y],[z]])
t1 = -R1 @ C

T1 = np.hstack((R1 , t1))

## Matriz intrínsica K1 = K2

file2 = 'data/DJI_0020.JPG'
GimbalYawDegree, GimbalPitchDegree, GimbalRollDegree, GPSLongitude, GPSLatitude, GPSAltitude, CalibratedFocalLength, CalibratedOpticalCenterX, CalibratedOpticalCenterY, FocalLength, ExifImageWidth, ExifImageHeight, FieldOfView = get_params(file2)

## Matriz extrínsica T2
R2 = Rypr(GimbalYawDegree, GimbalPitchDegree, GimbalRollDegree)
x, y, z = transformer.transform(GPSLatitude, GPSLongitude, GPSAltitude)  # C in world coordinates
C = np.array([[x],[y],[z]])
t2 = -R2 @ C

T2 = np.hstack((R2 , t2))

## Matriz intrínsica K2 modelagem 1 # Erro plan: 8.82 Erro alt: 110.2
cx = CalibratedOpticalCenterX
cy = CalibratedOpticalCenterY
f = CalibratedFocalLength
K1 = K2 = np.array([[f,0,cx],[0,-f,cy],[0,0,1]])

## Matriz intrínsica K2 modelagem 2 # Eplan: 8 Ealt: 107
# cx = CalibratedOpticalCenterX
# cy = CalibratedOpticalCenterY
# w, h = ExifImageWidth, ExifImageHeight
# fov_vert_rad = np.deg2rad(FieldOfView)
# fy = 0.5 * h / np.tan(fov_vert_rad / 2)
# fx = fy * (w / h)
# K1 = K2 = np.array([[fx,0,cx],[0,-fy,cy],[0,0,1]])

## Matriz intrínsica K2 modelagem 3 # Eplan: 8 Ealt: 107
# cx = CalibratedOpticalCenterX
# cy = CalibratedOpticalCenterY
# w, h = ExifImageWidth, ExifImageHeight
# d = np.sqrt(w**2 + h**2)
# fov_diagonal_rad = np.deg2rad(FieldOfView)
# f = 0.5*d/np.tan(fov_diagonal_rad/2)
# fx = f * (w / d)
# fy = f * (h / d)
# K1 = K2 = np.array([[fx,0,cx],[0,-fy,cy],[0,0,1]])

## Matriz intrínsica K1 modelagem 4 # Eplan: 8 Ealt: 107
# cx = CalibratedOpticalCenterX
# cy = CalibratedOpticalCenterY
# w, h = ExifImageWidth, ExifImageHeight
# fov_hor_rad = np.deg2rad(FieldOfView)
# fx = 0.5 * w / np.tan(fov_hor_rad / 2)
# fy = fx * (h / w)
# K1 = K2 = np.array([[fx,0,cx],[0,-fy,cy],[0,0,1]])

## Matriz intrínsica K2 modelagem 5 # Eplan: 10 Ealt: 105
# cx = CalibratedOpticalCenterX
# cy = CalibratedOpticalCenterY
# w, h = ExifImageWidth, ExifImageHeight
# d = np.sqrt(w**2 + h**2)
# f = CalibratedFocalLength
# fx = f * (w / d)
# fy = f * (h / d)
# K1 = K2 = np.array([[fx,0,cx],[0,-fy,cy],[0,0,1]])

## Matriz intrínsica K2 modelagem 6 # Eplan: 20  Ealt: 113 
# cx = CalibratedOpticalCenterX
# cy = CalibratedOpticalCenterY
# w, h = ExifImageWidth, ExifImageHeight
# d = np.sqrt(w**2 + h**2)
# larg, alt, diag = 12.8, 9.6, 16 # <-- Tabela sensores CMOS 1"
# sx = larg/w
# sy = alt/h
# f = FocalLength
# fx = f / sx
# fy = f / sy
# K1 = K2 = np.array([[fx,0,cx],[0,-fy,cy],[0,0,1]])

np.set_printoptions(suppress=True)

print("t1:\n", t1.reshape(-1))
print("t2:\n", t2.reshape(-1))
print("K1:\n", K1)
print("T1:\n", T1)
print("K2:\n", K1)
print("T2:\n", T2)

# (iii) Cálculo das matrizes de projeção das coordenadas do espaço-objeto para o espaço-imagem ($P_1$ e $P_2$):
P1 = K1 @ T1
P1 = P1.astype(float)
P2 = K2 @ T2
P2 = P2.astype(float)
print("P1:\n", np.round(P1, 5))
print("P2:\n", np.round(P2, 5))

# (iv) Ajustamento
P1_1 = P1[0] # 1ª linha de P1
P1_2 = P1[1] # 2ª linha de P1
P1_3 = P1[2] # 3ª linha de P1

P2_1 = P2[0] # 1ª linha de P2
P2_2 = P2[1] # 2ª linha de P2
P2_3 = P2[2] # 3ª linha de P2
u_p1 , v_p1 = p1
u_p2 , v_p2 = p2

A = np.array([u_p1 * P1_3 - P1_1,
              v_p1 * P1_3 - P1_2,
              u_p2 * P2_3 - P2_1,
              v_p2 * P2_3 - P2_2] , dtype=float)

# print("A:\n", np.round(A, 5))

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(A.T @ A)

# # Find the index of the minimum eigenvalue
min_index = np.argmin(eigenvalues)

# # Get the minimum eigenvalue and corresponding eigenvector
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

P_calc = min_eigenvector/min_eigenvector[-1] # Normalizando o vetor
print("P_calculado:\n", P_calc)

# Valores esperados e erro do ajustamento 9657987.969  825044.809  63.901 (int(gcp.iloc[0]["E"]) , int(gcp.iloc[0]["N"])
P_esperado = np.array([gcp.iloc[0]["E"], gcp.iloc[0]["N"],  gcp.iloc[0]["Z"],  1.]).T
print("P_esperado:\n", P_esperado)
print("Erro planimétrico:\n", np.linalg.norm(P_calc[:2] - P_esperado[:2]))
print("Erro altimétrico:\n", P_calc[2] - P_esperado[2])

t1:
 [8405807.93004682 4755979.60439549 -825021.3497339 ]
t2:
 [8535989.34725179 4518143.69982902 -825040.25166739]
K1:
 [[ 3666.666504     0.        2432.      ]
 [    0.       -3666.666504  1824.      ]
 [    0.           0.           1.      ]]
T1:
 [[     -0.              -0.8703557        0.49242356 8405807.93004682]
 [      0.              -0.49242356      -0.8703557  4755979.60439549]
 [      1.               0.               0.         -825021.3497339 ]]
K2:
 [[ 3666.666504     0.        2432.      ]
 [    0.       -3666.666504  1824.      ]
 [    0.           0.           1.      ]]
T2:
 [[     -0.00081669      -0.88376563       0.4679291  8535989.34725179]
 [      0.00154246      -0.46792981      -0.88376428 4518143.69982902]
 [      0.99999848       0.               0.00174533 -825040.25166739]]
P1:
 [[ 2.43200000e+03 -3.19130408e+03  1.80555297e+03  2.88148425e+10]
 [ 1.82400000e+03  1.80555297e+03  3.19130408e+03 -1.89434301e+10]
 [ 1.00000000e+00  0.00000000e+00  0.000000